# Tutorial de PyDeequ - Creación de constraints a partir de un batch de datos

## Asegurarse de tener instalada la versión correcta de Spark y de Pandas.

In [1]:
import os
os.environ["SPARK_VERSION"] = "3.3"

In [2]:
#!pip install pandas==1.5

## Crear una sesión de Spark

In [3]:
import pandas as pd
from pyspark.sql import SparkSession
import sagemaker_pyspark
import pydeequ  

classpath = ":".join(sagemaker_pyspark.classpath_jars())

spark = (SparkSession.builder.config("spark.driver.extraClassPath", classpath)
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate()
    )

24/04/17 22:25:19 WARN Utils: Your hostname, fernando-IdeaPad-L340 resolves to a loopback address: 127.0.1.1; using 192.168.100.61 instead (on interface wlp0s20f3)
24/04/17 22:25:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/fernando/Descargas/Demo%20KT%20DQ/kt-env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fernando/.ivy2/cache
The jars for the packages stored in: /home/fernando/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eb01036e-bb25-4829-85a9-9de418bdd9e5;1.0
	confs: [default]
	found com.amazon.deequ#deequ;2.0.3-spark-3.3 in central
	found org.scala-lang#scala-reflect;2.12.10 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_2.12;1.

24/04/17 22:25:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Obtener sugerencias de constraints a partir de un batch de datos

In [4]:
df = pd.read_csv('Data/Example_batch_0.csv')
df

,name,email,phone,address,date_of_birth,salary,department,hire_date,employee_id
0,Randy Lopez,jennifermitchell@example.net,(794)898-7688x98566,"9071 David Oval\nPort Randallside, MT 73314",1962-10-04,33170.0,Sales,2022-04-04,309020488
1,Angelica Smith,patrick66@example.net,455-528-0615,"3694 Jose Haven\nSouth Richard, NE 60772",1970-10-19,95214.0,Marketing,2018-06-27,3910194
2,Austin Watson,bradydaniel@example.com,845.865.2527x14944,"255 Christopher Oval\nEast Ashleyside, DE 97810",1956-09-09,93976.0,IT,2022-01-09,271465087
3,Melissa Watkins,kennedyrodney@example.org,9017179117,"6651 Jennifer Way Suite 839\nNorth Ericton, KY...",1980-11-18,84527.0,Sales,2009-12-27,814
4,Kathryn Moore,taylormonique@example.com,973.995.7456,"77848 Jacob Ford\nNorth Calvin, DC 21528",2003-07-15,51295.0,Marketing,2001-06-02,724669
...,...,...,...,...,...,...,...,...,...
1995,Erika Bell,lauren50@example.net,412.425.7145x632,"86475 Sarah Corners\nLeeland, IN 09806",2003-01-15,83994.0,HR,2019-05-11,15
1996,Frank Rose,valerie45@example.org,+1-422-400-5901,"4559 Hudson Trail Apt. 602\nJamiefurt, MP 43209",1976-07-23,53672.0,Finance,2001-10-26,9077394
1997,Ian Andrews,andersonbrandi@example.com,001-866-696-4010x500,"8081 Barry Squares\nNew Paul, VI 14898",1993-05-19,36887.0,Marketing,2018-10-13,942
1998,Christine Shelton,yaustin@example.org,598.262.0034x26222,"782 Johnson Port\nPort Melindabury, OK 32086",1997-03-18,69627.0,Sales,2023-06-04,989351


In [5]:
from pydeequ.suggestions import *  

suggestionResult = ConstraintSuggestionRunner(spark) \
    .onData(df) \
    .addConstraintRule(DEFAULT()) \
    .run()
suggestionResult

/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pydeequ/pandas_utils.py:26: UserWarning: WARNING: You passed in a Pandas DF, so we will be using our experimental utility to convert it to a PySpark DF.
  warnings.warn(
/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


24/04/17 22:25:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


{'constraint_suggestions': [{'constraint_name': 'CompletenessConstraint(Completeness(employee_id,None))',
   'column_name': 'employee_id',
   'current_value': 'Completeness: 1.0',
   'description': "'employee_id' is not null",
   'suggesting_rule': 'CompleteIfCompleteRule()',
   'rule_description': 'If a column is complete in the sample, we suggest a NOT NULL constraint',
   'code_for_constraint': '.isComplete("employee_id")'},
  {'constraint_name': "ComplianceConstraint(Compliance('employee_id' has no negative values,employee_id >= 0,None))",
   'column_name': 'employee_id',
   'current_value': 'Minimum: 0.0',
   'description': "'employee_id' has no negative values",
   'suggesting_rule': 'NonNegativeNumbersRule()',
   'rule_description': 'If we see only non-negative numbers in a column, we suggest a corresponding constraint',
   'code_for_constraint': '.isNonNegative("employee_id")'},
  {'constraint_name': 'UniquenessConstraint(Uniqueness(List(employee_id),None))',
   'column_name': 

### Mostrar los constraints sugeridos (descripción)

In [6]:
for sugg in suggestionResult['constraint_suggestions']:
	print(f"Constraint suggestion for \'{sugg['column_name']}\': {sugg['description']}")

Constraint suggestion for 'employee_id': 'employee_id' is not null
Constraint suggestion for 'employee_id': 'employee_id' has no negative values
Constraint suggestion for 'employee_id': 'employee_id' is unique
Constraint suggestion for 'hire_date': 'hire_date' is not null
Constraint suggestion for 'name': 'name' is not null
Constraint suggestion for 'name': 'name' is unique
Constraint suggestion for 'email': 'email' is not null
Constraint suggestion for 'email': 'email' is unique
Constraint suggestion for 'department': 'department' has value range 'Marketing', 'Sales', 'IT', 'HR', 'Finance', 'R&D', 'Legal', 'Customer Support'
Constraint suggestion for 'department': 'department' is not null
Constraint suggestion for 'department': 'department' has value range 'Marketing', 'Sales', 'IT', 'HR', 'Finance' for at least 98.0% of values
Constraint suggestion for 'salary': 'salary' is not null
Constraint suggestion for 'salary': 'salary' has no negative values
Constraint suggestion for 'salary'

### Mostrar los constraints sugeridos (código)

In [7]:
for sugg in suggestionResult['constraint_suggestions']:
	print(f"{sugg['code_for_constraint']}")

.isComplete("employee_id")
.isNonNegative("employee_id")
.isUnique("employee_id")
.isComplete("hire_date")
.isComplete("name")
.isUnique("name")
.isComplete("email")
.isUnique("email")
.isContainedIn("department", ["Marketing", "Sales", "IT", "HR", "Finance", "R&D", "Legal", "Customer Support"])
.isComplete("department")
.isContainedIn("department", ["Marketing", "Sales", "IT", "HR", "Finance"], lambda x: x >= 0.98, "It should be above 0.98!")
.isComplete("salary")
.isNonNegative("salary")
.isUnique("salary")
.isComplete("date_of_birth")
.isUnique("date_of_birth")
.isComplete("address")
.isUnique("address")
.isComplete("phone")
.isUnique("phone")


## Cargar un nuevo batch de data

In [8]:
df1 = pd.read_csv('Data/Example_batch_1.csv')
df1.columns

Index(['name', 'email', 'phone', 'address', 'date_of_birth', 'salary',
       'department', 'hire_date', 'employee_id'],
      dtype='object')

## Crear un Check

In [9]:
from pydeequ.checks import Check, CheckLevel
from pydeequ.verification import VerificationSuite, VerificationResult

check = Check(spark, CheckLevel.Error, "Integrity checks")


## Añadir los constraints sugeridos al Check, y correrlo

In [10]:
checkResult = VerificationSuite(spark) \
.onData(df1) \
.addCheck(
check \
    .isComplete("employee_id")
    .isNonNegative("employee_id")
    .isUnique("employee_id")
    .isComplete("hire_date")
    .isComplete("name")
    .isUnique("name")
    .isComplete("email")
    .isUnique("email")
    .isContainedIn("department", ["Marketing", "Sales", "IT", "HR", "Finance", "R&D", "Legal", "Customer Support"])
    .isComplete("department")
    .isContainedIn("department", ["Marketing", "Sales", "IT", "HR", "Finance"], lambda x: x >= 0.98, "It should be above 0.98!")
    .isComplete("salary")
    .isNonNegative("salary")
    .isUnique("salary")
    .isComplete("date_of_birth")
    .isUnique("date_of_birth")
    .isComplete("address")
    .isUnique("address")
    .isComplete("phone")
    .isUnique("phone")
).run()

/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pydeequ/pandas_utils.py:26: UserWarning: WARNING: You passed in a Pandas DF, so we will be using our experimental utility to convert it to a PySpark DF.
  warnings.warn(
/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Python Callback server started!


## Mostrar resultados

In [11]:
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.toPandas().head(10)

/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Integrity checks,Error,Error,CompletenessConstraint(Completeness(employee_i...,Success,
1,Integrity checks,Error,Error,ComplianceConstraint(Compliance(employee_id is...,Success,
2,Integrity checks,Error,Error,UniquenessConstraint(Uniqueness(List(employee_...,Success,
3,Integrity checks,Error,Error,"CompletenessConstraint(Completeness(hire_date,...",Success,
4,Integrity checks,Error,Error,"CompletenessConstraint(Completeness(name,None))",Success,
5,Integrity checks,Error,Error,"UniquenessConstraint(Uniqueness(List(name),None))",Failure,Value: 0.9775 does not meet the constraint req...
6,Integrity checks,Error,Error,"CompletenessConstraint(Completeness(email,None))",Success,
7,Integrity checks,Error,Error,"UniquenessConstraint(Uniqueness(List(email),No...",Failure,Value: 0.9785 does not meet the constraint req...
8,Integrity checks,Error,Error,ComplianceConstraint(Compliance(department con...,Success,
9,Integrity checks,Error,Error,CompletenessConstraint(Completeness(department...,Success,


In [12]:
result_df_pandas = checkResult_df.toPandas()
result_df_pandas

,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Integrity checks,Error,Error,CompletenessConstraint(Completeness(employee_i...,Success,
1,Integrity checks,Error,Error,ComplianceConstraint(Compliance(employee_id is...,Success,
2,Integrity checks,Error,Error,UniquenessConstraint(Uniqueness(List(employee_...,Success,
3,Integrity checks,Error,Error,"CompletenessConstraint(Completeness(hire_date,...",Success,
4,Integrity checks,Error,Error,"CompletenessConstraint(Completeness(name,None))",Success,
5,Integrity checks,Error,Error,"UniquenessConstraint(Uniqueness(List(name),None))",Failure,Value: 0.9775 does not meet the constraint req...
6,Integrity checks,Error,Error,"CompletenessConstraint(Completeness(email,None))",Success,
7,Integrity checks,Error,Error,"UniquenessConstraint(Uniqueness(List(email),No...",Failure,Value: 0.9785 does not meet the constraint req...
8,Integrity checks,Error,Error,ComplianceConstraint(Compliance(department con...,Success,
9,Integrity checks,Error,Error,CompletenessConstraint(Completeness(department...,Success,


In [14]:
for index, row in result_df_pandas.iterrows():
    if row.constraint_status != "Success":
        print(row.constraint)
        print(row.constraint_message)

UniquenessConstraint(Uniqueness(List(name),None))
Value: 0.9775 does not meet the constraint requirement!
UniquenessConstraint(Uniqueness(List(email),None))
Value: 0.9785 does not meet the constraint requirement!
UniquenessConstraint(Uniqueness(List(salary),None))
Value: 0.9625 does not meet the constraint requirement!
UniquenessConstraint(Uniqueness(List(date_of_birth),None))
Value: 0.9295 does not meet the constraint requirement!


## Anomaly Detection

In [16]:
from pyspark.sql import SparkSession, Row, DataFrame
import pandas as pd
import sagemaker_pyspark


import os
os.environ["SPARK_VERSION"] = "3.3"

import pydeequ

classpath = ":".join(sagemaker_pyspark.classpath_jars())

spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())
sc = spark.sparkContext

In [17]:
from pydeequ.repository import *
metricsRepository = InMemoryMetricsRepository(spark)

In [18]:
from pydeequ.verification import *
from pydeequ.analyzers import *
from pydeequ.anomaly_detection import *
import pandas as pd

yesterdaysDataset = pd.read_csv('Data/Example_batch_2.csv')
yesterdaysDataset=yesterdaysDataset.dropna()
yesterdaysDataset

,name,email,phone,address,date_of_birth,salary,department,hire_date,employee_id
0,Ann Mercer,patrickhunter@example.org,317.336.5702x36852,"PSC 1184, Box 8618\nAPO AP 61147",1974-07-11,80731.0,Sales,2017-01-29,1434
1,Steven Wiley,cruzhayley@example.com,201-990-5574x232,"6560 Camacho Fall Apt. 788\nVictoriaside, RI 2...",2004-02-12,86280.0,Marketing,2000-01-07,118016
2,Andrew Chavez,slowe@example.org,001-763-753-8897x1816,"28317 Morales Cove\nTownsendborough, ND 26280",1998-05-25,76932.0,IT,2004-06-10,38454784
3,James Ruiz,jasoncalderon@example.net,931-670-5728x818,"5021 Lucas Gardens\nNew Angelafurt, ID 08197",1945-05-02,71063.0,Finance,2017-09-09,4112
4,Danielle Golden,hunternicole@example.com,433-601-5477,"5632 Davis Green\nLake Ryan, KY 44464",1976-07-15,38614.0,Finance,2005-09-26,31127
...,...,...,...,...,...,...,...,...,...
1995,Jeffrey Woods,lawrenceshannon@example.org,8589384827,"1508 Aaron Curve\nLisashire, KY 66211",1957-03-07,88857.0,Finance,2003-06-13,65856
1996,Jennifer Ross,courtneysmith@example.net,(516)827-4193x364,"43659 Latasha Field\nSweeneyburgh, DC 40870",1998-11-30,95331.0,HR,2010-05-28,26076
1997,Glenda Herrera,robertadkins@example.com,457-518-9822x57915,"3416 Griffith Roads\nAmandastad, MP 92006",2004-10-30,76362.0,IT,2012-08-21,24309
1998,Christine Boyer,kimpaige@example.com,+1-918-297-6718x5402,"980 Robbins Squares\nSusanburgh, NM 02680",2003-03-29,72250.0,Marketing,2014-04-18,60561727


In [19]:
yesterdaysKey = ResultKey(spark, ResultKey.current_milli_time() - 24 * 60 * 60 * 1000)

prev_Result = VerificationSuite(spark).onData(yesterdaysDataset) \
    .useRepository(metricsRepository) \
    .saveOrAppendResult(yesterdaysKey) \
    .addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateIncrease=1.0), Size()) \
    .addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateDecrease=1.0), Size()) \
    .addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateIncrease=1.5), Mean("salary")) \
    .addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateDecrease=1.5), Mean("salary")) \
    .run()

/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pydeequ/pandas_utils.py:26: UserWarning: WARNING: You passed in a Pandas DF, so we will be using our experimental utility to convert it to a PySpark DF.
  warnings.warn(
/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [20]:
print(prev_Result.checkResultsAsDataFrame(spark,prev_Result).show())
print(prev_Result.checkResultsAsDataFrame(spark,prev_Result).toPandas().loc[:,"constraint_message"])

+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|               check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|Anomaly check for...|    Warning|     Warning|AnomalyConstraint...|          Failure|Can't execute the...|
|Anomaly check for...|    Warning|     Warning|AnomalyConstraint...|          Failure|Can't execute the...|
|Anomaly check for...|    Warning|     Warning|AnomalyConstraint...|          Failure|Can't execute the...|
|Anomaly check for...|    Warning|     Warning|AnomalyConstraint...|          Failure|Can't execute the...|
+--------------------+-----------+------------+--------------------+-----------------+--------------------+

None
0    Can't execute the assertion: requirement faile...
1    Can't execute the assertion: requirement faile...
2    Can't execute t

/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [21]:
todaysDataset = pd.read_csv('Data/Example_batch_3.csv')
todaysDataset.salary = todaysDataset.salary*4
todaysDataset = todaysDataset.loc[0:99]
todaysDataset

,name,email,phone,address,date_of_birth,salary,department,hire_date,employee_id
0,Michael Howard,james66@example.net,001-747-878-2015x0584,USS Myers\nFPO AA 16328,1958-08-31,371576.0,Finance,2002-01-18,38007
1,Daniel Ward,ycummings@example.net,935.245.9453x9843,"546 Gardner Well Suite 544\nHeatherport, VT 57835",1998-12-19,245548.0,Marketing,2018-07-09,57328683
2,Jennifer Russell,jmccoy@example.com,408.709.5148,Unit 1744 Box 2955\nDPO AE 26097,1992-04-22,131768.0,Marketing,2021-05-02,589651
3,Hayden Crawford,aalvarez@example.com,(581)219-5836x5753,"0280 Smith Mews\nDanielhaven, PA 28739",1961-06-22,137720.0,Marketing,2019-06-08,218602
4,Jackson Graham,jenniferwashington@example.net,814-587-1880,"5374 Rosario Hill Apt. 622\nNorth Megan, WV 83365",1996-05-23,322540.0,HR,2005-02-16,86783
...,...,...,...,...,...,...,...,...,...
95,Philip Webb,markrose@example.com,987-581-4137,"0458 Melissa Circle\nPetermouth, CO 41304",1991-10-03,348608.0,Marketing,2014-08-05,380262
96,Jennifer Barnes,amandawashington@example.com,(254)552-6942,"5980 Gibson Square\nWest Briantown, PW 76419",1976-04-10,217784.0,Sales,2010-02-19,415455363
97,Daniel Martin,luis97@example.net,718-298-9930,"611 Kathy Points Apt. 571\nNorth Samuel, ME 42731",1975-08-09,208208.0,IT,2004-01-02,103295
98,Miranda White,thill@example.org,(576)446-5648x0242,"59518 Denise Stream\nLake Diana, OH 50458",1960-06-13,234860.0,Sales,2011-02-28,33895699


In [22]:
todaysKey = ResultKey(spark, ResultKey.current_milli_time())

currResult = VerificationSuite(spark).onData(todaysDataset) \
    .useRepository(metricsRepository) \
    .saveOrAppendResult(todaysKey) \
    .addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateIncrease=1.0), Size()) \
    .addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateDecrease=1.0), Size()) \
    .addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateIncrease=1.5), Mean("salary")) \
    .addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateDecrease=1.5), Mean("salary")) \
    .run()

/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pydeequ/pandas_utils.py:26: UserWarning: WARNING: You passed in a Pandas DF, so we will be using our experimental utility to convert it to a PySpark DF.
  warnings.warn(
/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [27]:
print(currResult.checkResultsAsDataFrame(spark,currResult).show())
print(currResult.checkResultsAsDataFrame(spark,currResult).toPandas().loc[:,"constraint"])
print(currResult.checkResultsAsDataFrame(spark,currResult).toPandas().loc[:,"constraint_message"])

+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|               check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|Anomaly check for...|    Warning|     Success|AnomalyConstraint...|          Success|                    |
|Anomaly check for...|    Warning|     Warning|AnomalyConstraint...|          Failure|Value: 100.0 does...|
|Anomaly check for...|    Warning|     Warning|AnomalyConstraint...|          Failure|Value: 261440.24 ...|
|Anomaly check for...|    Warning|     Success|AnomalyConstraint...|          Success|                    |
+--------------------+-----------+------------+--------------------+-----------------+--------------------+

None


/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [24]:
pandas_df_results= currResult.checkResultsAsDataFrame(spark,currResult).toPandas()

for index, row in pandas_df_results.iterrows():
    if row.constraint_status != "Success":
        print(row.constraint)
        print(row.constraint_message) 

AnomalyConstraint(Size(None))
Value: 100.0 does not meet the constraint requirement!
AnomalyConstraint(Mean(salary,None))
Value: 261440.24 does not meet the constraint requirement!


/home/fernando/Descargas/Demo KT DQ/kt-env/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [25]:
print(yesterdaysDataset["salary"].mean())
print(todaysDataset["salary"].mean())

64603.61499235084
261440.24


In [26]:
if (currResult.status != "Success"):
    print("Anomaly detected")
    metricsRepository.load().forAnalyzers([Size()]).getSuccessMetricsAsDataFrame().show()
    metricsRepository.load().forAnalyzers([Mean("salary")]).getSuccessMetricsAsDataFrame().show()


Anomaly detected
+-------+--------+----+------+-------------+
| entity|instance|name| value| dataset_date|
+-------+--------+----+------+-------------+
|Dataset|       *|Size|1961.0|1713146354525|
|Dataset|       *|Size| 100.0|1713232755503|
+-------+--------+----+------+-------------+

+------+--------+----+-----------------+-------------+
|entity|instance|name|            value| dataset_date|
+------+--------+----+-----------------+-------------+
|Column|  salary|Mean|64603.61499235084|1713146354525|
|Column|  salary|Mean|        261440.24|1713232755503|
+------+--------+----+-----------------+-------------+

